In [1]:
import numpy as np
import itertools
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import datasets

Load and preprocess data

In [32]:
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
train_path = 'aclImdb/test'
test_path = 'aclImdb/train'

train_texts = []
train_labels = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_path, label_type)
    for fname in os.listdir(dir_name):
        if fname.endswith('.txt'):
            with open(os.path.join(dir_name, fname), encoding='utf-8') as f:
                train_texts.append(f.read())
            if label_type == 'neg':
                train_labels.append(0)
            else:
                train_labels.append(1)

test_texts = []
test_labels = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_path, label_type)
    for fname in os.listdir(dir_name):
        if fname.endswith('.txt'):
            with open(os.path.join(dir_name, fname), encoding='utf-8') as f:
                test_texts.append(f.read())
            if label_type == 'neg':
                test_labels.append(0)
            else:
                test_labels.append(1)
train_texts = np.array(train_texts)
test_texts = np.array(test_texts)
vectorizer_train = CountVectorizer()
vectorizer_test = CountVectorizer(max_features=73822)
train_features_ = vectorizer_train.fit_transform(train_texts)
train_features =  train_features_.toarray()
train_labels =  np.array(train_labels)
test_features_ = vectorizer_test.fit_transform(test_texts)
test_features =  test_features_.toarray()
test_labels =  np.array(train_labels)

In [41]:

class GaussianNaiveBayes:
    def __init__(self,laplace_smoothing=1):
        self.prior = None
        self.likelihood = None
        self.laplace_smoothing = laplace_smoothing
    def fit(self, X, y):
        self.prior = np.zeros(2)
        self.likelihood = np.zeros((2, X.shape[1]))
        for label in [0, 1]:
            subset = X[y == label]
            self.prior[label] = subset.shape[0] / X.shape[0]
            self.likelihood[label, :] = (np.sum(subset, axis=0) + self.laplace_smoothing) / (np.sum(subset) + X.shape[1])
        return self
    def predict(self, X):
        yh = np.zeros(X.shape[0])
        for i, x in enumerate(X):
            prediction = []
            for label in [0, 1]:
                likelihood_sum = np.sum(np.log(self.likelihood[label, x == 1]))
                prior_log = np.log(self.prior[label])
                prediction.append(prior_log + likelihood_sum)
            yh[i] = np.argmax(prediction)
        return yh



In [37]:
def evaluate_acc(predicted, actual):
    return np.sum(actual == predicted)/predicted.shape[0]

In [42]:
gnb = GaussianNaiveBayes(laplace_smoothing=1.3)
a = gnb.fit(train_features, train_labels)
yh = a.predict(train_features)
print(evaluate_acc(yh, train_labels))

0.90364


In [40]:
yh = a.predict(test_features)
print(evaluate_acc(yh, test_labels))

0.4962
